In [18]:
pip install torch torchvision opencv-python numpy scikit-learn matplotlib

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.20.0 requires keras>=3.10.0, but you have keras 3.4.1 which is incompatible.


   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/241.3 MB 889.4 kB/s eta 0:04:31
   ---------------------------------------- 0.8/241.3 MB 878.5 kB/s eta 0:04:34
   ---------------------------------------- 1.0/241.3 MB 996.2 kB/s eta 0:04:02
   ---------------------------------------- 1.3/241.3 MB 1.1 MB/s eta 0:03:45
   ----------------------------------------

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

csv_path = "table_nor.csv"
batch_size = 8
epochs = 25
lr = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

label_to_id = {"normal": 0, "sun": 1, "night": 2, "kontr": 3}
id_to_label = {v: k for k, v in label_to_id.items()}

df = pd.read_csv(csv_path)
df = df.drop(["Unnamed: 0"], axis=1)
df["label_id"] = df["state"].map(label_to_id)

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label_id"], random_state=42)

print("OK")

class GlareDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row["path"]).convert("RGB")
        label = row["label_id"]
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_ds = GlareDataset(train_df, transform)
test_ds = GlareDataset(test_df, transform)

print("OK")

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size)

print("OK")

model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(label_to_id))
model = model.to(device)

print("OK")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

print("OK")

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Эпоха [{epoch+1}/{epochs}] | Потеря: {total_loss/len(train_loader):.4f}")

model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\n=== Отчёт ===")
print(classification_report(y_true, y_pred, target_names=id_to_label.values()))

torch.save(model.state_dict(), "glare_detector.pth")
print("\n OK")

OK
OK
OK
OK
OK


c:\Users\nikit\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nikit\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Эпоха [1/25] | Потеря: 1.2353
Эпоха [2/25] | Потеря: 0.8211
Эпоха [3/25] | Потеря: 0.6014
Эпоха [4/25] | Потеря: 0.4727
Эпоха [5/25] | Потеря: 0.3699
Эпоха [6/25] | Потеря: 0.2415
Эпоха [7/25] | Потеря: 0.2213
Эпоха [8/25] | Потеря: 0.1979
Эпоха [9/25] | Потеря: 0.1626
Эпоха [10/25] | Потеря: 0.1162
Эпоха [11/25] | Потеря: 0.0930
Эпоха [12/25] | Потеря: 0.1121
Эпоха [13/25] | Потеря: 0.1129
Эпоха [14/25] | Потеря: 0.0674
Эпоха [15/25] | Потеря: 0.0674
Эпоха [16/25] | Потеря: 0.0535
Эпоха [17/25] | Потеря: 0.0616
Эпоха [18/25] | Потеря: 0.0493
Эпоха [19/25] | Потеря: 0.0339
Эпоха [20/25] | Потеря: 0.0380
Эпоха [21/25] | Потеря: 0.0478
Эпоха [22/25] | Потеря: 0.0419
Эпоха [23/25] | Потеря: 0.0525
Эпоха [24/25] | Потеря: 0.0369
Эпоха [25/25] | Потеря: 0.0215

=== Отчёт ===
              precision    recall  f1-score   support

      normal       0.79      0.95      0.86        20
         sun       0.89      0.73      0.80        11
       night       0.89      0.80      0.84        10
  

In [64]:
def predict_from_csv_model(image_path, model_path="glare_detector.pth"):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    model = models.mobilenet_v3_small(pretrained=False)
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, 4)
    model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()

    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)

    classes = ["normal", "sun", "night", "kontrast"]
    print(f"🔍 Состояние табло: {classes[predicted.item()]}")
    return classes[predicted.item()]

print(predict_from_csv_model("C:/Users/nikit/Downloads/3_b9ut-lh.jpg", "glare_detector.pth"))

🔍 Состояние табло: normal
normal


In [ ]:
pip install telebot


   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBotAPI]
   ---------------------------------------- 0/2 [pyTelegramBo

In [ ]:
import os
import logging
from io import BytesIO
from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.nn.functional import softmax

import telebot

TOKEN = os.getenv("BOT_TOKEN", "8328774807:AAFiKpua50YHuTRZ-CK-Oqr1qppUUY_Hby8")
MODEL_PATH = os.getenv("GLARE_MODEL_PATH", "glare_detector.pth")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ID_TO_LABEL = {0: "Нормально", 1: "Засвет от солнца", 2: "Искажение ночью", 3: "Контраст"}

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO
)
logger = logging.getLogger(__name__)

bot = telebot.TeleBot(TOKEN)

_model = None

def load_model(model_path=MODEL_PATH, device=DEVICE):
    global _model
    if _model is not None:
        return _model

    model = models.mobilenet_v3_small(pretrained=False)
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(ID_TO_LABEL))
    state = torch.load(model_path, map_location="cpu")
    model.load_state_dict(state)
    model.to(device)
    model.eval()

    _model = model
    logger.info(f"✅ Model loaded from {model_path} on {device}")
    return _model

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

@bot.message_handler(commands=["start", "help"])
def start_message(message):
    bot.reply_to(message,"👋 Привет! Пришли фото табло, и я скажу — normal / sun / night / kontr.\n""Можно отправлять как фото или как файл.")

@bot.message_handler(content_types=["photo", "document"])
def handle_photo(message):
    try:
        model = load_model()

        file_info = None
        if message.photo:
            file_info = bot.get_file(message.photo[-1].file_id)
        elif message.document and message.document.mime_type.startswith("image/"):
            file_info = bot.get_file(message.document.file_id)
        else:
            bot.reply_to(message, "Пожалуйста, отправь изображение (jpg/png).")
            return

        downloaded = bot.download_file(file_info.file_path)
        image = Image.open(BytesIO(downloaded)).convert("RGB")

        input_tensor = transform(image).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            outputs = model(input_tensor)
            probs = softmax(outputs, dim=1).cpu().squeeze(0)
            confs = probs.tolist()

        top_idx = int(torch.argmax(probs).item())
        label = ID_TO_LABEL.get(top_idx, str(top_idx))
        confidence = confs[top_idx]

        file_id = message.photo[-1].file_id
        
        text = f"🔍 Предсказание: {label}\nВероятность: {confidence:.3f}\n\nВсе классы:\n"
        for i, p in enumerate(confs):
            text += f"- {ID_TO_LABEL.get(i, i)}: {p:.3f}\n"

        bot.send_photo(
                    chat_id=message.chat.id,
                    photo=file_id,
                    caption=text
                )

        
    except Exception as e:
        logger.exception("Error in handle_photo")
        bot.reply_to(message, e)

if __name__ == "__main__":
    logger.info("🚀 Бот запущен и ожидает сообщения...")
    bot.infinity_polling()
    

2025-10-16 17:20:55,742 - INFO - 🚀 Бот запущен и ожидает сообщения...
c:\Users\nikit\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nikit\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
2025-10-16 17:21:10,135 - INFO - ✅ Model loaded from glare_detector.pth on cpu


In [1]:
import requests

response = requests.get("https://api.telegram.org")
print(response.status_code)

200


In [4]:
import glob
import pandas as pd
import os

folder_path = 'nor\Обрезанные'

txt_files = glob.glob(os.path.join(folder_path, '*.png'))

df = pd.DataFrame(txt_files, columns=['path'])
df["state"] = "normal"

print(df)


                                                 path   state
0   nor\Обрезанные\225_ceed118c-6743-441b-a5ff-62e...  normal
1   nor\Обрезанные\226_d7166a4d-5f1f-4d07-a324-d8a...  normal
2   nor\Обрезанные\301_f90c1fd3-9e57-4561-8502-4ae...  normal
3   nor\Обрезанные\304_e048662d-7973-47aa-aa8f-667...  normal
4   nor\Обрезанные\440_cd72aba6-fdca-4247-9422-d5b...  normal
..                                                ...     ...
95  nor\Обрезанные\загруженное - 2025-09-30T163018...  normal
96  nor\Обрезанные\загруженное - 2025-09-30T163030...  normal
97  nor\Обрезанные\загруженное - 2025-10-02T103255...  normal
98  nor\Обрезанные\загруженное - 2025-10-02T103311...  normal
99  nor\Обрезанные\загруженное - 2025-10-02T103316...  normal

[100 rows x 2 columns]


In [5]:
folder_path = 'nor\Обрезанные\искажение ночь'

txt_files = glob.glob(os.path.join(folder_path, '*.png'))

df_night = pd.DataFrame(txt_files, columns=['path'])
df_night["state"] = "night"

print(df_night)

                                                 path  state
0   nor\Обрезанные\искажение ночь\154_05fb0ee5-586...  night
1   nor\Обрезанные\искажение ночь\75_48018a58-a07a...  night
2   nor\Обрезанные\искажение ночь\92_5694d77c-5d53...  night
3   nor\Обрезанные\искажение ночь\загружен123ное (...  night
4   nor\Обрезанные\искажение ночь\загружен124124но...  night
5   nor\Обрезанные\искажение ночь\загруженн1231ое ...  night
6   nor\Обрезанные\искажение ночь\загруженн124124о...  night
7   nor\Обрезанные\искажение ночь\загруженное (1) ...  night
8   nor\Обрезанные\искажение ночь\загруженное (10)...  night
9   nor\Обрезанные\искажение ночь\загруженное (12)...  night
10  nor\Обрезанные\искажение ночь\загруженное (13)...  night
11  nor\Обрезанные\искажение ночь\загруженное (14)...  night
12  nor\Обрезанные\искажение ночь\загруженное (15)...  night
13  nor\Обрезанные\искажение ночь\загруженное (16)...  night
14  nor\Обрезанные\искажение ночь\загруженное (18)...  night
15  nor\Обрезанные\искаж

In [6]:
folder_path = 'nor\Обрезанные\искажение солнце'

txt_files = glob.glob(os.path.join(folder_path, '*.png'))

df_sun = pd.DataFrame(txt_files, columns=['path'])
df_sun["state"] = "sun"

print(df_sun)

                                                 path state
0   nor\Обрезанные\искажение солнце\118_0299e275-a...   sun
1   nor\Обрезанные\искажение солнце\118_2a2cce65-2...   sun
2   nor\Обрезанные\искажение солнце\118_300552be-3...   sun
3   nor\Обрезанные\искажение солнце\118_57e8938e-b...   sun
4   nor\Обрезанные\искажение солнце\118_839d93ca-2...   sun
5   nor\Обрезанные\искажение солнце\118_956130d8-2...   sun
6   nor\Обрезанные\искажение солнце\118_b49b0a8c-a...   sun
7   nor\Обрезанные\искажение солнце\118_b56005fc-2...   sun
8   nor\Обрезанные\искажение солнце\123_38cf50a8-3...   sun
9   nor\Обрезанные\искажение солнце\123_4f6a3774-3...   sun
10  nor\Обрезанные\искажение солнце\123_8952c7bb-2...   sun
11  nor\Обрезанные\искажение солнце\123_95ad8006-7...   sun
12  nor\Обрезанные\искажение солнце\123_9a46abeb-d...   sun
13  nor\Обрезанные\искажение солнце\123_cc673900-e...   sun
14  nor\Обрезанные\искажение солнце\123_cf11d3c7-5...   sun
15  nor\Обрезанные\искажение солнце\123_

In [7]:
folder_path = 'nor\Обрезанные\контраст'

txt_files = glob.glob(os.path.join(folder_path, '*.png'))

df_kontr = pd.DataFrame(txt_files, columns=['path'])
df_kontr["state"] = "kontr"

print(df_kontr)

                                                 path  state
0   nor\Обрезанные\контраст\124_1956e2d8-59a8-4191...  kontr
1   nor\Обрезанные\контраст\124_340c1a1f-2474-44c8...  kontr
2   nor\Обрезанные\контраст\124_5408e7d3-be29-4f7b...  kontr
3   nor\Обрезанные\контраст\124_62912d7e-5280-4b78...  kontr
4   nor\Обрезанные\контраст\124_9ff322df-bc03-45ad...  kontr
..                                                ...    ...
73   nor\Обрезанные\контраст\загруженное (43) (2).png  kontr
74   nor\Обрезанные\контраст\загруженное (44) (1).png  kontr
75   nor\Обрезанные\контраст\загруженное (86) (1).png  kontr
76  nor\Обрезанные\контраст\загруженное - 2025-10-...  kontr
77  nor\Обрезанные\контраст\загруженное - 2025-10-...  kontr

[78 rows x 2 columns]


In [8]:
df = pd.concat([df, df_sun])
df = pd.concat([df, df_night])
df = pd.concat([df, df_kontr])

print(df)

                                                 path   state
0   nor\Обрезанные\225_ceed118c-6743-441b-a5ff-62e...  normal
1   nor\Обрезанные\226_d7166a4d-5f1f-4d07-a324-d8a...  normal
2   nor\Обрезанные\301_f90c1fd3-9e57-4561-8502-4ae...  normal
3   nor\Обрезанные\304_e048662d-7973-47aa-aa8f-667...  normal
4   nor\Обрезанные\440_cd72aba6-fdca-4247-9422-d5b...  normal
..                                                ...     ...
73   nor\Обрезанные\контраст\загруженное (43) (2).png   kontr
74   nor\Обрезанные\контраст\загруженное (44) (1).png   kontr
75   nor\Обрезанные\контраст\загруженное (86) (1).png   kontr
76  nor\Обрезанные\контраст\загруженное - 2025-10-...   kontr
77  nor\Обрезанные\контраст\загруженное - 2025-10-...   kontr

[285 rows x 2 columns]


In [9]:
df.to_csv("table_nor.csv")
df = pd.read_csv("table_nor.csv")
df = df.drop(["Unnamed: 0"], axis=1)
print(df)

                                                  path   state
0    nor\Обрезанные\225_ceed118c-6743-441b-a5ff-62e...  normal
1    nor\Обрезанные\226_d7166a4d-5f1f-4d07-a324-d8a...  normal
2    nor\Обрезанные\301_f90c1fd3-9e57-4561-8502-4ae...  normal
3    nor\Обрезанные\304_e048662d-7973-47aa-aa8f-667...  normal
4    nor\Обрезанные\440_cd72aba6-fdca-4247-9422-d5b...  normal
..                                                 ...     ...
280   nor\Обрезанные\контраст\загруженное (43) (2).png   kontr
281   nor\Обрезанные\контраст\загруженное (44) (1).png   kontr
282   nor\Обрезанные\контраст\загруженное (86) (1).png   kontr
283  nor\Обрезанные\контраст\загруженное - 2025-10-...   kontr
284  nor\Обрезанные\контраст\загруженное - 2025-10-...   kontr

[285 rows x 2 columns]
